<a href="https://colab.research.google.com/github/colinmcnamara/3D-Printing-STEAM-curriculum/blob/master/Agents_Combined_with_vector_stores_as_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Combine agents and vector stores
This notebook covers how to combine agents and vectorstores. The use case for this is that you've ingested your data into a vectorstore and want to interact with it in an agentic manner.

The recommended method for doing so is to create a RetrievalQA and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vectordbs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vectorstores as normal tools, or you can set return_direct=True to really just use the agent as a router.

In [43]:
!pip install -q langchain openai colab_env chromadb tiktoken

In [44]:
import colab_env
import langchain
import openai
import os

In [45]:
openai_api_key=os.environ['OPENAI_API_KEY']
serpapi_api_key=os.environ['SERPAPI_API_KEY']

In [46]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

llm = OpenAI(temperature=0)

In [47]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "/content/gdrive/MyDrive/state_of_the_union.txt")

In [48]:
from langchain.document_loaders import TextLoader

loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [49]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [50]:
from langchain.document_loaders import WebBaseLoader

In [51]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [52]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

#create the agent

In [53]:
# Import things that are needed generically
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper

In [54]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [55]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [56]:
agent.run(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I need to find out what Biden said about Ketanji Brown Jackson in the State of the Union address.
Action: State of Union QA System
Action Input: What did Biden say about Ketanji Brown Jackson in the State of the Union address?
Observation:  Biden said that he had nominated Circuit Court of Appeals Judge Ketanji Brown Jackson four days ago, and that she was one of the nation's top legal minds who would continue Justice Breyer's legacy of excellence.
Thought: I now know the final answer
Final Answer: Biden said that he had nominated Circuit Court of Appeals Judge Ketanji Brown Jackson four days ago, and that she was one of the nation's top legal minds who would continue Justice Breyer's legacy of excellence.

> Finished chain.


"Biden said that he had nominated Circuit Court of Appeals Judge Ketanji Brown Jackson four days ago, and that she was one of the nation's top legal minds who would continue Justice Breyer's legacy of excellence."

In [57]:
agent.run("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 I need to find out the advantages of using ruff over flake8
Action: Ruff QA System
Action Input: What are the advantages of using ruff over flake8?
Observation:  Ruff can minimize conflicts across plugins and allows any individual plugin to be toggled on or off with a single (e.g.) --select TID, as opposed to --select I2 (to avoid conflicts with the isort rules, like I001). Ruff also replaces isort, yesqa, eradicate, and most of the rules implemented in pyupgrade. Additionally, Ruff is installable under any Python version from 3.7 onwards.
Thought: I now know the final answer
Final Answer: Ruff can minimize conflicts across plugins and allows any individual plugin to be toggled on or off with a single (e.g.) --select TID, as opposed to --select I2 (to avoid conflicts with the isort rules, like I001). Ruff also replaces isort, yesqa, eradicate, and most of the rules implemented in pyupgrade. Additionally, Ruff is installable under any Python vers

'Ruff can minimize conflicts across plugins and allows any individual plugin to be toggled on or off with a single (e.g.) --select TID, as opposed to --select I2 (to avoid conflicts with the isort rules, like I001). Ruff also replaces isort, yesqa, eradicate, and most of the rules implemented in pyupgrade. Additionally, Ruff is installable under any Python version from 3.7 onwards.'

#Use the Agent solely as a router
You can also set return_direct=True if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [58]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [59]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [60]:
agent.run(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I need to find out what Biden said about Ketanji Brown Jackson in the State of the Union address.
Action: State of Union QA System
Action Input: What did Biden say about Ketanji Brown Jackson in the State of the Union address?
Observation:  Biden said that he had nominated Circuit Court of Appeals Judge Ketanji Brown Jackson four days ago, and that she was one of the nation's top legal minds who would continue Justice Breyer's legacy of excellence.


> Finished chain.


" Biden said that he had nominated Circuit Court of Appeals Judge Ketanji Brown Jackson four days ago, and that she was one of the nation's top legal minds who would continue Justice Breyer's legacy of excellence."

#Multi-Hop vectorstore reasoning
Because vectorstores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vectorstores using the existing agent framework

In [61]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [62]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [63]:
agent.run(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
 I need to find out what tool ruff uses to run over Jupyter Notebooks, and if the president mentioned it in the state of the union.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  nbQA
Thought: I need to find out if the president mentioned nbQA in the state of the union.
Action: State of Union QA System
Action Input: Did the president mention nbQA in the state of the union?
Observation:  No, the president did not mention nbQA in the state of the union.
Thought: I now know the final answer.
Final Answer: Ruff uses nbQA to run over Jupyter Notebooks, but the president did not mention nbQA in the state of the union.

> Finished chain.


'Ruff uses nbQA to run over Jupyter Notebooks, but the president did not mention nbQA in the state of the union.'